In [25]:
import os 
from dotenv import load_dotenv
load_dotenv()
from langchain_groq import ChatGroq
os.getenv("GROQ_API_KEY")
groq_llm=ChatGroq(model="Mixtral-8x7b-32768")

In [26]:
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY")

# SYSTEM ,  HUMAN AND AI MESSAGE TEMPLATE

In [27]:
from langchain_core.prompts import PromptTemplate , ChatPromptTemplate , SystemMessagePromptTemplate , HumanMessagePromptTemplate 

one_input_prompt=PromptTemplate.from_template("You are a helpful assitant that helps in {field}")
formatted_prompt=one_input_prompt.format(field="real estate")
response=groq_llm.invoke(formatted_prompt)
print("Response is " , response)

Response is  content="Hello! I'm here to help answer any questions you might have about real estate. Whether you're a first-time homebuyer, a seasoned investor, or just curious about the market, I'm here to assist you.\n\nHere are a few ways I can help you:\n\n1. Provide information on the current real estate market trends, including home prices, sales data, and inventory levels.\n2. Assist with the home buying or selling process, including researching properties, scheduling showings, and negotiating offers.\n3. Connect you with local real estate agents, mortgage brokers, and other professionals who can help you navigate the home buying or selling process.\n4. Answer questions about the home buying or selling process, including what to expect, how to prepare, and what documents you'll need.\n5. Provide tips and advice on how to make the most of your real estate investment, including how to increase your home's value, how to stage your home for sale, and how to negotiate the best deal.\

In [28]:
sys_template = "You are a helpful {role_associate} assistant"
human_template= "{input}"

new_chat_prompt=ChatPromptTemplate.from_messages([

    SystemMessagePromptTemplate.from_template(sys_template),
    HumanMessagePromptTemplate.from_template(human_template)
])

print(new_chat_prompt)

formatted_chats=new_chat_prompt.format_messages(role_associate = "Real Estate" , input="I have 2 million dollars what will be the best Roi in real estate and at which place in us")

groq_llm.invoke(formatted_chats)

input_variables=['input', 'role_associate'] messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['role_associate'], template='You are a helpful {role_associate} assistant')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}'))]


AIMessage(content='The best Return on Investment (ROI) in real estate can vary based on the current market conditions, but generally, buying a property that can be rented out or "flipped" for a profit can provide a good ROI.\n\nOne popular strategy is to invest in residential rental properties, such as single-family homes or small apartment buildings, in areas with a strong rental market and a growing job market. Some cities that currently have a strong rental market and a healthy job market include:\n\n* Raleigh, North Carolina\n* Austin, Texas\n* Charlotte, North Carolina\n* Denver, Colorado\n* Nashville, Tennessee\n\nAnother strategy is to invest in fixer-upper properties, also known as "fix and flip" properties. These properties can be purchased at a lower price, then renovated and sold for a profit. Popular markets for fix and flip properties include:\n\n* Atlanta, Georgia\n* Phoenix, Arizona\n* Las Vegas, Nevada\n* Tampa, Florida\n* Cleveland, Ohio\n\nIt\'s important to note that

# OUTPUT PARSERS 


In [29]:
from langchain.output_parsers import PydanticOutputParser 
from langchain_core.pydantic_v1 import BaseModel , Field

class Crickter(BaseModel):
    vectorestore:str = Field(description="Crickter name")
    record : list = Field(description="The record that the crickter has made so far in a python list")

parser=PydanticOutputParser(pydantic_object=Crickter)

new_system_template="You are a hard core cricket fan and you have all the knowlegde about the cricket world"
human_template="{input} \n {formatted_output}"

chat_prompt_template=ChatPromptTemplate.from_messages([SystemMessagePromptTemplate.from_template(new_system_template) , HumanMessagePromptTemplate.from_template(human_template)])
formatting_template=chat_prompt_template.format_messages(input="Which is the best cricketer in the world?" , formatted_output=parser.get_format_instructions())


print(formatting_template)

response=groq_llm.invoke(formatting_template)
print("Response : "  ,response)


[SystemMessage(content='You are a hard core cricket fan and you have all the knowlegde about the cricket world'), HumanMessage(content='Which is the best cricketer in the world? \n The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"vectorestore": {"title": "Vectorestore", "description": "Crickter name", "type": "string"}, "record": {"title": "Record", "description": "The record that the crickter has made so far in a python list", "type": "array", "items": {}}}, "required": ["vectorestore", "record"]}\n```')]
Response :  content='Here is an example of a well-formatted JS

In [30]:
print("Formatted_response" , parser.parse(response.content))

Formatted_response vectorestore='Sachin Tendulkar' record=[15921, 18122, 1553, 1715, 14692, 1648, 18040, 1562, 1583, 15965, 15810, 1564, 1478, 15474, 15641, 15534, 1523, 16008, 15947, 15737]


## Create React Agent Prompt Manually

In [31]:
from langchain.agents import AgentExecutor, create_tool_calling_agent

from langchain_core.prompts import PromptTemplate

template = '''Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}'''

react_prompt = PromptTemplate.from_template(template)

second_prompt_template=prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant named ABAX"),
        ("human", "{input}"),
        # Placeholders fill up a **list** of messages
        ("placeholder", "{agent_scratchpad}"),
    ]
)

# prompt_template_for_langgraph_react_agent =  ChatPromptTemplate.from_messages([
#     ...     ("system", "You are a helpful bot named Fred."),
#     ...     ("placeholder", "{messages}"),
#     ...     ("user", "Remember, always be polite!"),
#     ... ])
# def modify_messages(messages: list):
#     return prompt_template_for_langgraph_react_agent.invoke({"messages": messages})


## Custom Tools Creation 

In [32]:
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain.tools import BaseTool, StructuredTool, tool
from langchain import hub
from langchain_core.prompts import PromptTemplate
from langchain_core.tools import Tool
from langchain.agents import create_react_agent, AgentExecutor
# from langgraph.prebuilt import create_react_agent
from langchain_groq import ChatGroq
from langchain_core.tools import ToolException
from typing import Type
load_dotenv()
os.getenv("GROQ_API_KEY")
# Define the CalculatorInput schema
class CalculatorInput(BaseModel):
    """It is used to define the input of the Calculator tool."""
    a: int = Field(description="first number to multiply")
    b: int = Field(description="second number to multiply")

@tool
def multiply(a: int, b: int) -> int:
    """Multiply two numbers."""
    return a * b

def _handle_error(error: ToolException) -> str:
    return (
        "The following errors occurred during tool execution:"
        + error.args[0]
        + "Please try another tool."
    )

calculator = StructuredTool.from_function(
    func=multiply,
    name="Calculator",
    description="multiply numbers",
    args_schema=CalculatorInput,
    return_direct=False,
    handle_tool_error=_handle_error

    # coroutine= ... <- you can specify an async method if desired as well
)


# Pull the prompt from the hub
# prompt = hub.pull("hwchase17/react")

# Initialize the language model
@tool
def magic_fuction(input:int)->int:
    """Magic Function , this will be used to apply magic to a number"""
    return input+2

@tool
def investment_function(input3:str)->str:
    """This Function will take in the user price and then convert into dollars"""
    investment_response=groq_llm.invoke("Convert this price in user query to usd dollars"+input3)
    return investment_response.content
# Define the tools
tools = [ calculator ,  magic_fuction , investment_function]

# system_message="You are a real estate assitant named ABAX and you work for ABAX estates that helps in giving the best investment advises for real esate that will have highest ROI for the investement amount user tells . Suggest them best investments that do with that amount in the cities of that country break it region wise .If the user requests for our human service representatives they can contact us on out on  email which is abaxesates@gmail.com and our phone number is 03123456789 this is where the user can also contact us"


prompt=hub.pull("hwchase17/openai-tools-agent")

agent=create_tool_calling_agent(llm=groq_llm , tools=tools , prompt=prompt)
agent_executor=AgentExecutor.from_agent_and_tools(agent=agent , tools=tools , verbose=True , handle_parsing_errors=True)
reponse=agent_executor.invoke({"input":"What is 5 Multiplied 3 equals to ??"})
print(response.content)






> Entering new AgentExecutor chain...

Invoking: `Calculator` with `{'a': 5, 'b': 3}`




TypeError: BaseTool.__call__() got an unexpected keyword argument 'a'

In [ ]:
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain.tools import BaseTool, StructuredTool, tool
from langchain import hub
from langchain_core.prompts import PromptTemplate
from langchain_core.tools import Tool
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain_groq import ChatGroq
from langchain_core.tools import ToolException
from typing import Type
import os
from dotenv import load_dotenv

load_dotenv()
groq_llm = ChatGroq(api_key=os.getenv("GROQ_API_KEY"))

# Define the CalculatorInput schema
class CalculatorInput(BaseModel):
    """It is used to define the input of the Calculator tool."""
    a: int = Field(description="first number to multiply")
    b: int = Field(description="second number to multiply")

@tool
def multiply(a: int, b: int) -> int:
    """Multiply two numbers."""
    return a * b

def _handle_error(error: ToolException) -> str:
    return (
        "The following errors occurred during tool execution:"
        + error.args[0]
        + "Please try another tool."
    )

calculator = StructuredTool.from_function(
    func=multiply,
    name="Calculator",
    description="multiply numbers",
    args_schema=CalculatorInput,
    return_direct=False,
    handle_tool_error=_handle_error
)

@tool
def magic_function(input: int) -> int:
    """Magic Function, this will be used to apply magic to a number"""
    return input + 2

@tool
def investment_function(input3: str) -> str:
    """This Function will take in the user price and then convert into dollars"""
    investment_response = groq_llm.invoke("Convert this price in user query to usd dollars" + input3)
    return investment_response.content

# Define the tools
tools = [calculator, magic_function, investment_function]

prompt = hub.pull("hwchase17/openai-tools-agent")

agent = create_tool_calling_agent(llm=groq_llm, tools=tools, prompt=prompt)
agent_executor = AgentExecutor.from_agent_and_tools(agent=agent, tools=tools, verbose=True, handle_parsing_errors=True)
response = agent_executor.invoke({"input": "What is 5 Multiplied 3 equals to ??"})
print(response.content)




> Entering new AgentExecutor chain...

Invoking: `Calculator` with `{'a': 5, 'b': 3}`




TypeError: BaseTool.__call__() got an unexpected keyword argument 'a'

In [37]:
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain.tools import BaseTool, StructuredTool, tool
from langchain import hub
from langchain_core.prompts import PromptTemplate
from langchain_core.tools import Tool
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain_groq import ChatGroq
from langchain_core.tools import ToolException
from typing import Type
import os
from dotenv import load_dotenv

load_dotenv()

# Define the CalculatorInput schema
class CalculatorInput(BaseModel):
    """It is used to define the input of the Calculator tool."""
    a: int = Field(description="first integer number")
    b: int = Field(description="second integer number")

class MultiplyNumbersTool(BaseTool):
    name = "Multiply numbers"
    description = "Useful for multiplying two numbers"
    args_schema: Type[BaseModel] = CalculatorInput

    def _run(self, a: int, b: int) -> str:
        """Use the tool."""
        result = a * b
        return f"The product of {a} and {b} is {result}"

@tool
def magic_function(input: int) -> int:
    """Magic Function, this will be used to apply magic to a number."""
    return input + 2

@tool
def investment_function(input3: str) -> str:
    """This Function will take in the user price and then convert it into dollars."""
    investment_response = groq_llm.invoke("Convert this price in user query to usd dollars" + input3)
    return investment_response.content

# Instantiate the tools
multiply_tool = MultiplyNumbersTool()
tools = [multiply_tool, magic_function, investment_function]

# Pull the prompt from the hub
prompt = hub.pull("hwchase17/openai-tools-agent")

agent = create_tool_calling_agent(llm=groq_llm, tools=tools, prompt=prompt)
agent_executor = AgentExecutor.from_agent_and_tools(agent=agent, tools=tools, verbose=True, handle_parsing_errors=True)
response = agent_executor.invoke({"input": "What is 5 into 3 equals to ??"})
print(response.content)
 



> Entering new AgentExecutor chain...
</tool-use>{"tool_call":{"id":"pending","type":"function","function":{"name":"Multiply numbers"},"parameters":{"a":5,"b":3}}}</tool-use>

> Finished chain.


AttributeError: 'dict' object has no attribute 'content'

In [ ]:
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain.tools import BaseTool, StructuredTool, tool
from langchain import hub
from langchain_core.prompts import PromptTemplate
from langchain_core.tools import Tool
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain_groq import ChatGroq
from langchain_core.tools import ToolException
from typing import Type
import os
from dotenv import load_dotenv

load_dotenv()
groq_llm = ChatGroq(api_key=os.getenv("GROQ_API_KEY"))

# Define the CalculatorInput schema
class CalculatorInput(BaseModel):
    """It is used to define the input of the Calculator tool."""
    a: int = Field(description="first integer number")
    b: int = Field(description="second integer number")

class MultiplyNumbersTool(BaseTool):
    name = "Multiply numbers"
    description = "Useful for multiplying two numbers"
    args_schema: Type[BaseModel] = CalculatorInput

    def _run(self, a: int, b: int) -> str:
        """Use the tool."""
        result = a * b
        return f"The product of {a} and {b} is {result}"

@tool
def magic_function(input: int) -> int:
    """Magic Function, this will be used to apply magic to a number."""
    return input + 2

@tool
def investment_function(input3: str) -> str:
    """This Function will take in the user price and then convert it into dollars."""
    investment_response = groq_llm.invoke("Convert this price in user query to usd dollars" + input3)
    return investment_response.content

# Instantiate the tools
multiply_tool = MultiplyNumbersTool()
tools = [multiply_tool, magic_function, investment_function]

# Pull the prompt from the hub
prompt = hub.pull("hwchase17/openai-tools-agent")

agent = create_tool_calling_agent(llm=groq_llm, tools=tools, prompt=prompt)
agent_executor = AgentExecutor.from_agent_and_tools(agent=agent, tools=tools, verbose=True, handle_parsing_errors=True)
response = agent_executor.invoke({"input": "What is 5 multiplied 3 equals to??"})
print(response.content)




> Entering new AgentExecutor chain...

Invoking: `Multiply numbers` with `{'a': 5, 'b': 3}`


The product of 5 and 3 is 15The product of 5 and 3 is 15.

> Finished chain.


AttributeError: 'dict' object has no attribute 'content'

In [ ]:
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain.tools import BaseTool, StructuredTool, tool
from langchain import hub
from langchain_core.prompts import PromptTemplate
from langchain.agents import create_react_agent, AgentExecutor
from langchain_groq import ChatGroq
from langchain_core.tools import ToolException



load_dotenv()
os.getenv("GROQ_API_KEY")
# Define the CalculatorInput schema
class CalculatorInput(BaseModel):
    """It is used to define the input of the Calculator tool."""
    a: int = Field(description="first integer number")
    b: int = Field(description="second integer number")



def _handle_error(error: ToolException) -> str:
    return (
        "The following errors occurred during tool execution:"
        + error.args[0]
        + "Please try another tool."
    )

calculator = StructuredTool.from_function(
    func=multiply,
    name="Calculator",
    description="multiply numbers",
    args_schema=CalculatorInput,
    return_direct=False,
    handle_tool_error=_handle_error

    # coroutine= ... <- you can specify an async method if desired as well
)

# Pull the prompt from the hub
prompt = hub.pull("hwchase17/openai-tools-agent")
# Initialize the language model

@tool
def multiply(a: int, b: int) -> int:
    """Multiply two numbers."""
    return a * b


# Define the tools
tools = [multiply]

# # Create the agent using the custom prompt
agent = create_tool_calling_agent(groq_llm, tools=tools , prompt=second_prompt_template)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True , handle_parsing_errors=True)


# Example invocation
response = agent_executor.invoke({"input": "What is 5 times 3"})
print(response['output'])



> Entering new AgentExecutor chain...

Invoking: `multiply` with `{'a': 5, 'b': 3}`


15Fifteen.

> Finished chain.
Fifteen.


In [ ]:
prompt_for_function_calling=ChatPromptTemplate.from_messages([("system" , "You are a personal assistant that helps with the user quries") , ("placeholder" , "{chat_history}") , ("human" , "{input}") , ("placeholder" , "{agent_scratchpad}")])


In [ ]:
from langchain_community.document_loaders import TextLoader 
from langchain.text_splitter import  RecursiveCharacterTextSplitter , CharacterTextSplitter
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain_chroma import Chroma
from langchain.tools import BaseTool, StructuredTool, tool
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults()


directory_path = "./documents"

text_files = [f for f in os.listdir(directory_path) if f.endswith('.txt')]
if text_files:
    for text_file in text_files:
        file_path = os.path.join(directory_path, text_file)
        loader = TextLoader(file_path)
        my_data=loader.load()
        splitter = CharacterTextSplitter(separator="\n", chunk_size=1000, chunk_overlap=200)
        chunks = splitter.split_text(my_data)

        model_name = "BAAI/bge-small-en"
        model_kwargs = {"device": "cpu"}
        encode_kwargs = {"normalize_embeddings": True}
        hf = HuggingFaceBgeEmbeddings(
            model_name=model_name, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs
        )   
        vectore_store = Chroma.from_documents(documents=chunks , collection_name='zilbil_store' , embedding=hf)







@tool
def pdf_query(pdf_input:str)->str:
    """This will return the results of the user query from the vector store"""
    docs = vectore_store.similarity_search(pdf_input)
    return print(docs[0].page_content)

@tool
def websearch_tool(web_input:str)->str:
    """This will search the user query on the internet"""
    custom_content=[]
    web_response=search.run(web_input)
    for result in web_response:
        web_content=result.get('content' , '')
        if web_content:
            custom_content.append(web_content)
    return custom_content



tools=[pdf_query , websearch_tool]
